In [1]:

# For the widest path problem, we need a Max-Heap, rather than a Min-Heap. 
# We use some techniques to convert a Min-Heap into a Max-Heap.




In [2]:
exec(open("./funcs/tool_funcs.py").read())




In [3]:
 

def dijkstra_one_to_all_widest_path(distance_matrix, src, need_up_i, remove_node):
    
    N = len(distance_matrix)
 
    dist = [0] * N
    dist[src] = np.inf
    visited = [False] * N
 
    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
    pq = [(-np.inf, src)]
    
    kk = len(need_up_i)
    counter = 0
    visited[remove_node] = True

    while pq:
        current_dist, u = heapq.heappop(pq)
        # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
        current_dist = -current_dist

        if visited[u]:
            continue
        visited[u] = True
        
        if u in need_up_i:
            counter += 1
            if counter == kk:
                
                return np.array(dist)
 

        # Update distances for neighbors
        for v in range(N):
            if not visited[v] and distance_matrix[u][v] < np.inf:
                new_dist = min(current_dist, distance_matrix[u][v])
                if new_dist > dist[v]:
                    dist[v] = new_dist
                    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
                    heapq.heappush(pq, (-new_dist, v))

    return np.array(dist)



In [4]:
def cal_need_update_list_widest_path(remove_node, X_widest_path_matrix):
    N = len(X_widest_path_matrix)

    removal = remove_node
    remaining_list = [i for i in np.arange(N) if i != removal]
    need_update_list = [[] for i in range(N)]
    how_many_counter = np.zeros(N)
    for i in remaining_list:
        for j in remaining_list:
            if i != j:
                if X_widest_path_matrix[i,j] <= min(X_widest_path_matrix[i,removal], X_widest_path_matrix[removal,j]):
                    need_update_list[i].append(j)
                    how_many_counter[i] += 1
    
    how_many = np.sum(how_many_counter > 0)
    
    
    return need_update_list, how_many

In [5]:
def generate_new_distance_matrix_widest_path(X_distance_matrix, remove_node):
    X_distance_matrix_new = X_distance_matrix.copy()
 

    N = len(X_distance_matrix)
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_distance_matrix_new[i,j] = 0

    X_distance_matrix_new[remove_node,remove_node] = np.inf  
    
    return X_distance_matrix_new

In [6]:

def cal_new_widest_path_matrix_warm(X_distance_matrix, X_widest_path_matrix, remove_node):
    N = len(X_distance_matrix)
    X_widest_path_matrix_new = X_widest_path_matrix.copy()
    remaining_list = [i for i in np.arange(N) if i != remove_node]

    need_update_list, how_many = cal_need_update_list_widest_path(remove_node, X_widest_path_matrix)

 
    Cost = np.round(how_many/(N - 1), 3)
    
    print(f"The cost for re-caculating the widest path matrix is: {Cost}.")   
    
    
    X_distance_matrix_new = generate_new_distance_matrix_widest_path(X_distance_matrix, remove_node)
 
    for i in remaining_list:
        if len(need_update_list[i]) > 0:
            temp = dijkstra_one_to_all_widest_path(X_distance_matrix_new, i, need_update_list[i], remove_node)
            for j in need_update_list[i]:
                X_widest_path_matrix_new[i,j] = temp[j]
 
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_widest_path_matrix_new[i,j] = 0

    X_widest_path_matrix_new[remove_node,remove_node] = np.inf
    
    return X_widest_path_matrix_new 


In [7]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)


In [8]:
remove_node = 3 

X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")
 

In [9]:
# For the widest path problem, a node's bandwidth to itself should be infinity.


N = len(X_distance_matrix)
for i in range(N):
    X_distance_matrix[i,i] = np.inf




In [10]:
X_widest_path_matrix = floyd_warshall_widest_path(X_distance_matrix)

X_distance_matrix_new = generate_new_distance_matrix_widest_path(X_distance_matrix, remove_node)



In [11]:

start = time.time()
X_widest_path_matrix_new_warm = cal_new_widest_path_matrix_warm(X_distance_matrix, X_widest_path_matrix, remove_node)
end = time.time()


time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

print(f"Time used: {time_used1_warm}s" )

The cost for re-caculating the widest path matrix is: 1.0.
Time used: 0.4s


In [12]:

start = time.time()
X_widest_path_matrix_new_floyd_warshall = floyd_warshall_widest_path(X_distance_matrix_new)
end = time.time()


time_used2_floyd = end - start
time_used2_floyd = np.round(time_used2_floyd, 3)

print(f"Time used: {time_used2_floyd}s" )

Time used: 1.153s


In [13]:
print(f"Ratio: {np.round(time_used1_warm/time_used2_floyd, 2)}")


Ratio: 0.35


In [14]:
print(np.allclose(X_widest_path_matrix_new_warm, X_widest_path_matrix_new_floyd_warshall))


True


In [15]:
X_widest_path_matrix_new_warm[2]

array([989., 980.,  inf,   0., 982., 985., 987., 989., 982., 989., 975.,
       987., 989., 983., 950., 985., 986., 989., 984., 984., 981., 987.,
       989., 976., 986., 989., 987., 989., 977., 989., 960., 982., 989.,
       983., 985., 986., 987., 987., 976., 981., 982., 984., 986., 989.,
       970., 987., 972., 995., 987., 989., 989., 988., 986., 976., 980.,
       988., 986., 988., 973., 987., 987., 987., 988., 985., 997., 989.,
       985., 975., 983., 984., 980., 986., 989., 976., 986., 989., 984.,
       989., 989., 981., 986., 989., 980., 988., 982., 987., 965., 987.,
       986., 986., 975., 989., 989., 983., 989., 987., 989., 984., 990.,
       987.])